In [3]:
### Testing reset

from four_room.env import FourRoomsEnv
from four_room.wrappers import gym_wrapper
from four_room.utils import obs_to_state, obs_to_img
import gymnasium as gym
import dill

gym.register('MiniGrid-FourRooms-v1', FourRoomsEnv)

with open('./four_room/configs/fourrooms_train_config.pl', 'rb') as file:
    train_config = dill.load(file)
        #(player location x, player location y, player direction, goal location x, goal location y, 
         #   door position up, door position down, door position left, door position right)


def describe(state):
    state=obs_to_state(state)
    print("----Observation----")
    print(f"Player: {state[0]},{state[1]}, dir: {state[2]}")
    print(f"Goal: {state[3]},{state[4]}")
    print(f"Doors: up:{state[5]}, down:{state[6]}, left:{state[7]}, right:{state[8]}")



def make_env_fn(config, seed: int= 0, rank: int = 0):
    def _init():
        env = gym_wrapper(gym.make('MiniGrid-FourRooms-v1', 
                    agent_pos=config['agent positions'], 
                    goal_pos=config['goal positions'], 
                    doors_pos=config['topologies'], 
                    agent_dir=config['agent directions']))
        env.reset(seed=seed+rank)
        return env

    return _init

env = make_env_fn(train_config,seed=123958)()
target_env = make_env_fn(train_config,seed=412318)()

target, _ = target_env.reset()
obs,_ = env.reset(options={'load_state':target})
target2, _, _ , _ , _  = target_env.step(2)
obs2,  _ ,_ ,_ ,_  = env.step(2)

print("----To mimic: ----")
describe(target)
describe(target2)

print("----Reconstruction: ---")

describe(obs)
describe(obs2)



----To mimic: ----
----Observation----
Player: 5,5, dir: 1
Goal: 2,1
Doors: up:0, down:1, left:1, right:1
----Observation----
Player: 5,6, dir: 1
Goal: 2,1
Doors: up:0, down:1, left:1, right:1
----Reconstruction: ---
----Observation----
Player: 5,5, dir: 1
Goal: 3,7
Doors: up:2, down:1, left:0, right:1
----Observation----
Player: 5,6, dir: 1
Goal: 3,7
Doors: up:2, down:1, left:0, right:1


/home/felix/.local/lib/python3.10/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment MiniGrid-FourRooms-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
